In [2]:
import skorch
from datajoint_tables import *
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
from plotting import *
from sklearn.linear_model import Ridge
from tdr import *
from Regressor import *
from Trials import *
from RNN import *
import pickle
from fit_dpca import *
from Perturbation import Perturbation
from LatentCircuitAnalysis import *
%matplotlib tk

Connecting langdon@pioneer.cshl.edu:3306


# Models

In [56]:
Model() 

model_id unique model id,connectivity specify connectivity structure,n number of neurons,lr learning rate,batch_size,patience,threshold,lambda_r firing rate regularization constant,lambda_o orthogonality regularization constant,lambda_w weight regularization,r2,valid_loss,train_loss,epochs,l2_ortho,l2_rate,l2_weight,l2_task,w_rec,w_in,w_out,embedding,train_loss_history,valid_loss_history
0QkxckGk,large,150,0.001000,128,100,0.00010,0.000000,0.000000,0.000000,0.966944,0.00516626,0.00595381,1314,2.63798,0.304468,0.0244929,0.00516626,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3kOtND7Y,large,150,0.010000,128,5,0.00010,0.050000,1.000000,0.000000,0.922854,0.012941,0.0116437,69,0.0,0.292172,0.0229329,0.0120571,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
3ZUOEIuF,large,150,0.010000,128,25,0.00010,0.000000,1.000000,0.000000,0.945626,0.00851028,0.00742974,215,1.22829e-05,0.302799,0.0335701,0.00849799,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
7GqynGFx,large,150,0.010000,128,100,0.00010,0.000000,0.000000,0.000000,0.986541,0.00210353,0.00553989,973,1.99717,0.311128,0.0688043,0.00210353,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
fZ27AusU,large,150,0.010000,128,100,0.00010,0.000000,0.000000,0.000000,0.981404,0.00290628,0.00278682,604,2.37395,0.303972,0.0397839,0.00290628,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
g4mvhTL4,large,150,0.001000,128,100,0.00010,0.050000,0.000000,0.000000,0.973592,0.00459443,0.00337027,1303,2.68655,0.306733,0.0249514,0.00412731,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
hyFO11uN,large,150,0.010000,128,50,0.00010,0.050000,1.000000,0.000000,0.98009,0.00368991,0.00503728,488,4.97844e-05,0.304208,0.0664296,0.00311164,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
Qj40HgX7,large,150,0.010000,128,25,0.00010,0.050000,1.000000,0.000000,0.967655,0.00566079,0.00872359,100,9.23914e-08,0.317202,0.0265928,0.00505518,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
qxOSUDtj,large,150,0.010000,128,100,0.00010,0.050000,1.000000,0.000000,0.955632,0.00799956,0.00712602,731,0.00045239,0.312728,0.10385,0.00693419,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
TIZjKvTA,large,150,0.001000,128,100,0.00010,0.000000,0.000000,0.000000,0.980334,0.00307357,0.00491538,838,2.85968,0.305179,0.0191416,0.00307357,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


# Model reductions

In [99]:
pd.DataFrame((LCA() & 'model_id="3ZUOEIuF"' ).proj('model_id','lca_id','sigma_rec','lr','batch_size','patience','epochs','valid_loss','train_loss','n_trials').fetch()).sort_values('valid_loss',ascending=True)

,model_id,lca_id,sigma_rec,lr,patience,n_trials,batch_size,epochs,valid_loss,train_loss
143,3ZUOEIuF,sB2NI1Tp,0.15,0.0100000,50,64,128,1132,0.12339,0.12233
27,3ZUOEIuF,8P6mC7D1,0.15,0.0100000,50,64,128,584,0.12538,0.13578
195,3ZUOEIuF,zabOLlxl,0.15,0.0100000,50,64,128,599,0.12647,0.13056
50,3ZUOEIuF,clcMMw41,0.15,0.0100000,50,64,128,563,0.12818,0.12856
99,3ZUOEIuF,L7jqoMq3,0.15,0.0100000,50,64,128,708,0.12843,0.12806
...,...,...,...,...,...,...,...,...,...,...
38,3ZUOEIuF,Aqafn5IH,0.15,0.0100000,50,64,128,289,0.26038,0.26026
109,3ZUOEIuF,m4rz5JmZ,0.15,0.0100000,50,64,128,183,0.26114,0.21752
19,3ZUOEIuF,5Bfrjn5Z,0.15,0.0100000,50,64,128,265,0.26194,0.24887
182,3ZUOEIuF,Xl10Rm0W,0.15,0.0100000,50,64,128,297,0.26787,0.26842


In [55]:
LCA_unconstrained().delete()

About to delete:
`langdon_rnn`.`l_c_a_unconstrained`: 1 items


Proceed? [yes, No]:  yes


Committed.


In [90]:
loss = (Model() & 'model_id="x9Yv2etv"').fetch1('valid_loss_history')
plt.figure()
plt.plot(loss)
plt.ylim(0,1)

(0.0, 1.0)

In [27]:
LCA()

model_id unique model id,lca_id,alpha,sigma_rec,lr,weight_decay,patience,threshold,n_trials,batch_size,max_epochs,epochs,r2_x,r2_xqt,r2_z,valid_loss,train_loss,valid_loss_history,train_loss_history,w_rec,w_in,w_out,q,a,w_error,q_error
x9Yv2etv,0fnPbLKi,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,224,0.0659,0.1329,0.0838,0.28260,0.29129,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,15QKwKRr,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,279,0.0583,0.0681,0.0454,0.17188,0.17304,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,1gtBy3ts,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,459,0.0551,0.0562,0.0359,0.14721,0.14596,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,1kZ3KgKz,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,341,0.0556,0.0603,0.0468,0.16272,0.15187,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,3327O0NU,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,360,0.0613,0.0952,0.0560,0.21258,0.21659,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,5jovkyYL,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,359,0.0562,0.0572,0.0392,0.15250,0.15070,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,BKVGD1eJ,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,359,0.0577,0.0605,0.0460,0.16425,0.16436,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,br0O2UZW,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,445,0.0601,0.1258,0.0344,0.22032,0.22299,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,EFWPl46t,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,644,0.0563,0.0999,0.0398,0.19604,0.20388,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan
x9Yv2etv,fDeLRaWE,0.20,0.15,0.0100000,0.00000,50,0.0001000,64,128,20000,307,0.0546,0.0523,0.0362,0.14307,0.15087,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,nan,nan


# Figure: Connectivity

In [91]:
model_id = "3ZUOEIuF"
lca_id = "zabOLlxl"
linewidth=1.5
fig = plt.figure(figsize=(20,4))
gs = gridspec.GridSpec(1,5, figure=fig,width_ratios=[1.5,.2,1.5,1.5,1.5])

# RNN connectivity
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])
ax4 = fig.add_subplot(gs[4])

w_rec = (Model() & {'model_id':  model_id}).fetch1('w_rec')
w_in = (Model() & {'model_id':  model_id}).fetch1('w_in')
w_out = (Model() & {'model_id':  model_id}).fetch1('w_out')
A=np.zeros((158,158))
A[6:156,:6]=w_in
A[6:156,6:156]=w_rec
A[156:,6:156]=w_out
sns.heatmap(A,center=0,ax=ax0,xticklabels=False,yticklabels=False,cbar=False)
ax0.axvline(x=6, color='w',linewidth=linewidth)
ax0.axvline(x=156, color='w',linewidth=linewidth)
ax0.axhline(y=6, color='w',linewidth=linewidth)
ax0.axhline(y=156, color='w',linewidth=linewidth)
ax0.set_title(r'$W$')

q = (LCA() & {'lca_id':  lca_id}).fetch1('q')
sns.heatmap(q.T,center=0,ax=ax1,xticklabels=False,yticklabels=False,cbar=False)
ax1.set_title(r'$Q$')
# ax1.axvline(x=1, color='w',linewidth=linewidth)
# ax1.axvline(x=2, color='w',linewidth=linewidth)
# ax1.axvline(x=3, color='w',linewidth=linewidth)
# ax1.axvline(x=4, color='w',linewidth=linewidth)
# ax1.axvline(x=5, color='w',linewidth=linewidth)
# ax1.axvline(x=6, color='w',linewidth=linewidth)
# ax1.axvline(x=7, color='w',linewidth=linewidth)


w_rec = (LCA() & {'lca_id':  lca_id}).fetch1('w_rec')
w_in = (LCA() & {'lca_id':  lca_id}).fetch1('w_in')
w_out = (LCA() & {'lca_id': lca_id}).fetch1('w_out')
A=np.zeros((16,16))
A[6:14,:6]=w_in
A[6:14,6:14]=w_rec
A[14:,6:14]=w_out
sns.heatmap(A,center=0,ax=ax2,xticklabels=False,yticklabels=False,cbar=False)
ax2.set_title(r'$\bar{W}$')
ax2.axvline(x=6, color='w',linewidth=linewidth)
ax2.axvline(x=14, color='w',linewidth=linewidth)
ax2.axhline(y=6, color='w',linewidth=linewidth)
ax2.axhline(y=14, color='w',linewidth=linewidth)

b,_ = np.linalg.qr(q.T,mode='complete')
w_rec = (Model() & {'model_id':  model_id}).fetch1('w_rec')
b[:,:8] = q.T
w = -b.T @ w_rec @ b
sns.heatmap(-w,center=0,ax=ax3,xticklabels=False,yticklabels=False,cbar=False)
ax3.set_title(r'$\mathcal{O}^TW_{rec}\mathcal{O}$')

sns.heatmap(-w[:8,:8], center=0, xticklabels=False,yticklabels=False,ax=ax4)
ax4.axhline(y=0, color='w',linewidth=4)
ax4.set_title(r'$\mathcal{Q}^TW_{rec}\mathcal{Q}$')
plt.savefig('/users/langdon/figures/circuitpaperfigures/results_1.png',dpi=1000, transparent=True)

# Figure: Dynamics

In [93]:
model_id = "3ZUOEIuF"
lca_id = "zabOLlxl"
fig = plt.figure(figsize=(8,3))
gs = gridspec.GridSpec(2,4, figure=fig)
# RNN connectivity

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax3 = fig.add_subplot(gs[2])
ax4 = fig.add_subplot(gs[3])

q = (LCA() & {'lca_id':lca_id}).fetch1('q')
q =  q[[0,2,4,6],:] - q[[1,3,5,7],:]
    
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Load inputs and labels
inputs, labels, mask, conditions = generate_trials(**trial_events,
                                          n_trials=15,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=6)


n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))

dt=1
z_mask, z, x = rnn.forward(inputs)
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
x = np.reshape(x,(-1,x.shape[2])) @ q.T
z = np.diff(np.reshape(z,(-1,z.shape[2])))

conditions = np.repeat(pd.DataFrame(conditions).values, repeats=n_t // dt, axis=0)

data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x','motion_x','color_x', 'choice_x','z'])

df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["context_x"] = df["context_x"].astype(float)
df["motion_x"] = df["motion_x"].astype(float)
df["color_x"] = df["color_x"].astype(float)
df["choice_x"] = df["choice_x"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df["z"] = df["z"].astype(float)

 # Plotting
fontsize = 6
lw=1

sns.lineplot(
data=df,
x="time",
y="context_x",
style='context',
color='gray',
legend=False,
ax = ax1,
    ci='sd',
    lw=lw
)
ax1.axvspan(40*trial_events['cue_on'], 40*trial_events['cue_off'], alpha=0.5, color='lightgray')
ax1.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.5, color='lightgray')
ax1.axvspan(40*trial_events['dec_on'], 40*trial_events['dec_off'], alpha=0.5, color='lightgray')
for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
    label.set_fontsize(fontsize)
ax1.set_xticklabels([])


ax1.set_ylabel("")
#ax1.tick_params(bottom=False)
#ax1.tick_params(left=False)
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)
#plt.legend(loc='upper left')

#Motion 

palette = sns.color_palette("coolwarm", 6)
sns.lineplot(
data=df,
x="time",
y="motion_x",
hue='motion_coh',
style='context',
palette=palette,
legend=False,
    ax = ax2,
ci='sd',
    lw=lw
)
ax2.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.5, color='lightgray')


for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
    label.set_fontsize(fontsize)
ax2.set_xticklabels([])
ax2.set_ylabel("")
ax2.tick_params(bottom=False)
ax2.tick_params(left=False)
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

#plt.legend(loc='upper left')

# Color 

palette = sns.color_palette("coolwarm", 6)
sns.lineplot(
data=df,
x="time",
y="color_x",
hue = 'color_coh',
style='context',
palette=palette,
legend=False,
    ax = ax3,
    ci='sd',
    lw=lw
)
ax3.axvspan(40*trial_events['stim_on'], 40*trial_events['stim_off'], alpha=0.5, color='lightgray')
for label in (ax3.get_xticklabels() + ax3.get_yticklabels()):
    label.set_fontsize(fontsize)

ax3.set_ylabel("")
ax3.set_xticklabels([])
ax3.tick_params(bottom=False)
ax3.tick_params(left=False)
ax3.spines["top"].set_visible(False)
ax3.spines["right"].set_visible(False)

#plt.legend(loc='upper left')

# Choice

palette = sns.color_palette("coolwarm", 2)
sns.lineplot(
data=df,
x="time",
y="choice_x",
hue = 'correct_choice',
style='context',
palette=palette,
legend=False,
    ax = ax4,
    ci='sd',
    lw=lw
)
ax4.axvspan(40*trial_events['dec_on'], 40*trial_events['dec_off'], alpha=0.5, color='lightgray')
for label in (ax4.get_xticklabels() + ax4.get_yticklabels()):
    label.set_fontsize(fontsize)

ax4.set_ylabel("")
ax4.set_xticklabels([])
ax4.tick_params(bottom=False)
ax4.tick_params(left=False)
ax4.spines["top"].set_visible(False)
ax4.spines["right"].set_visible(False)


for ax in [ax1,ax2,ax3,ax4]:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(fontsize)
    ax.set_xticklabels([])
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)





#PLOT FITS AND 2D PROJECTIONS
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Load inputs and labels
inputs, labels, mask, conditions = generate_trials(**trial_events,
                                          n_trials=15,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=4)

n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))
z_mask, z, x = rnn.forward(inputs)
dt=1
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
repeats= x.shape[1]
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
q = (LCA() & {'lca_id':lca_id}).fetch1('q')
x = np.reshape(x,(-1,x.shape[2])) @ q.T
z = np.reshape(z,(-1,z.shape[2]))
conditions = np.repeat(pd.DataFrame(conditions).values, repeats=repeats, axis=0)

data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])
df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df1 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()


# Latent circuit activity
n = 8

rnn = RNNModule(connectivity='small', mask=mask, n=n,sigma_rec=.15)
rnn.recurrent_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_out'))

z_mask, z, x = rnn.forward(inputs)
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
x = np.reshape(x,(-1,x.shape[2])) 
z = np.reshape(z,(-1,z.shape[2]))
data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])

df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df2 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()

#fig = plt.figure(figsize=(18,3.5))

ax5 = fig.add_subplot(gs[4])
ax6 = fig.add_subplot(gs[5])
ax7 = fig.add_subplot(gs[6])
ax8 = fig.add_subplot(gs[7])

for label in (ax5.get_xticklabels() + ax5.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax6.get_xticklabels() + ax6.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax7.get_xticklabels() + ax7.get_yticklabels()):
    label.set_fontsize(fontsize)
for label in (ax8.get_xticklabels() + ax8.get_yticklabels()):
    label.set_fontsize(fontsize)


markersize=2
markevery=2
lw=1

# Context subspace
df= df1.groupby(['context','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()

cohs=np.linspace(-.2,.2,2)
ax5.plot(df[(df['context']=='motion') ]['context_x_1'],df[(df['context']=='motion')]['context_x_2'],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax5.plot(df[(df['context']=='color')]['context_x_1'],df[(df['context']=='color')]['context_x_2'],color='gray',lw=lw)

df= df2.groupby(['context','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
ax5.plot(df[(df['context']=='motion') ]['context_x_1'],df[(df['context']=='motion')]['context_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax5.plot(df[(df['context']=='color')]['context_x_1'],df[(df['context']=='color')]['context_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)

# Motion subspace
df = df1.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
cohs=np.linspace(-.2,.2,4)
for i in range(4):
    ax6.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax6.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
df = df2.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
for i in range(4):
    ax6.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax6.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)

# Color subspace
df = df1.groupby(['context','color_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
cohs=np.linspace(-.2,.2,4)
for i in range(4):
    ax7.plot((df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
    ax7.plot((df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
df = df2.groupby(['context','color_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
for i in range(4):
    ax7.plot((df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='motion') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax7.plot((df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_1'])[30:],(df[(df['context']=='color') & (df['color_coh']==cohs[i])]['color_x_2'])[30:],lw=lw,color='lightgray',marker='.',markersize=markersize,markevery=markevery)

# Choice subspace
df = df1.groupby(['correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()  
ax8.plot(df[(df['correct_choice']==1) ]['choice_x_1'],df[(df['correct_choice']==1)]['choice_x_2'],color='gray',marker='.',markersize=markersize,markevery=markevery,lw=lw)
ax8.plot(df[(df['correct_choice']==-1)]['choice_x_1'],df[(df['correct_choice']==-1)]['choice_x_2'],marker='.',markersize=markersize,markevery=markevery,color='gray',lw=lw)

df = df2.groupby(['correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()  
ax8.plot(df[(df['correct_choice']==1) ]['choice_x_1'],df[(df['correct_choice']==1)]['choice_x_2'],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
ax8.plot(df[(df['correct_choice']==-1)]['choice_x_1'],df[(df['correct_choice']==-1)]['choice_x_2'],lw=lw,color='lightgray',marker='.',markersize=markersize,markevery=markevery)


for ax in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8]:
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.tick_params(axis='both', which='major', pad=1)


plt.savefig('/users/langdon/figures/circuitpaperfigures/results_2.png',dpi=1000, transparent=True)

# Figure: Sloppiness

In [3]:
# loss surface along first two principal components
model_id = "3ZUOEIuF"
lca_id = "zabOLlxl"
w_recs = np.stack((LCA() & {'model_id':model_id}).fetch('w_rec')).reshape(-1,8*8)
w_ins = np.stack((LCA() & {'model_id':model_id} ).fetch('w_in')).reshape(-1,8*6)
w_outs = np.stack((LCA() & {'model_id':model_id}).fetch('w_out')).reshape(-1,2*8)
mean_q = np.mean((LCA() & {'model_id':model_id} ).fetch('q'))
ws = np.concatenate((w_ins,w_recs,w_outs),axis=1)


pca = PCA()
pca.fit(ws)


N = (Model() & {'model_id': model_id}).fetch1('n')
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}


# Load inputs and labels
n_trials = 25
inputs, _, mask, conditions = generate_trials(**trial_events,
                                          n_trials=n_trials,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=6)

n = (Model() & {'model_id':model_id}).fetch1('n')
rnn_net = RNNNet(
    module=RNNModule,
    module__n=n,
    module__connectivity='large',
    baseline=.015,
    module__mask = mask,
    device=device,
)
rnn_net.initialize()

rnn_net.module_.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'), device= device)
rnn_net.module_.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'),device=device)
rnn_net.module_.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'),device=device)

_, z, x = rnn_net.forward(inputs.to(device=device),training=False)
q_true = torch.tensor((Model() & {'model_id': model_id}).fetch1('embedding')).float()
x = x.detach().cpu() @ q_true
z = z.detach().cpu()
labels = torch.cat((x,z), dim=2)
# Load inputs and labels

recurrent_mask = torch.ones(8, 8).float().to(device=device)
recurrent_mask = recurrent_mask - torch.diag(torch.diag(recurrent_mask))
input_mask = torch.cat((torch.eye(6),torch.zeros(2,6)),dim=0)
output_mask = torch.cat((torch.zeros(2,6),torch.eye(2)),dim=1)
k=25
surface = np.zeros((k,k))
sloppy_lambdas = np.linspace(-.5,.5,k)
tight_lambdas = np.linspace(-.0001,.0001,k)
for i in range(k):
    for j in range(k):
        latent_net = LatentNet(
            module=LatentModule,
            module__n=8,
            module__N=N,
            module__alpha = 0.2,
            module__sigma_rec = .15,
            module__weight_decay=.001,
            module__recurrent_mask=recurrent_mask.to(device=device),
            module__input_mask=input_mask.to(device=device),
            module__output_mask=output_mask.to(device=device),
            warm_start=False,
            #lr=parameters['lr'],
            #batch_size=int(parameters['batch_size']),
            max_epochs=20000,
            optimizer=torch.optim.Adam,

        )
        latent_net.initialize()
        
        W =pca.inverse_transform(tight_lambdas[i] * pca.components_[0,:] + sloppy_lambdas[j] * pca.components_[10,:])
        
        W_in = W[:8*6].reshape(8,6)
        W_rec = W[8*6: 8*6 + 8*8].reshape(8,8)
        W_out = W[8*6 + 8*8:].reshape(2,8)


        latent_net.module_.input_layer.weight.data = torch.tensor(W_in)
        latent_net.module_.recurrent_layer.weight.data = torch.tensor(W_rec)
        latent_net.module_.output_layer.weight.data = torch.tensor(W_out)
        latent_net.module_.q = torch.tensor(mean_q)
        y_pred=latent_net.forward(inputs)
        surface[i,j]=latent_net.get_loss(y_pred,labels).detach().numpy()

#plt.savefig('/users/langdon/figures/circuitpaperfigures/results_3.png',dpi=1000, transparent=True)

In [6]:
fig = plt.figure(figsize=(8,1.5))
gs = gridspec.GridSpec(1,5, figure=fig,width_ratios=[1,1,1,1,1.2])
p=100
# RNN connectivity
ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])
ax4 = fig.add_subplot(gs[4])
df=pd.DataFrame((LCA() & {'model_id': model_id}).fetch())
df['valid_loss']=df['valid_loss'].astype(float)
df['train_loss']=df['train_loss'].astype(float)

data=[]
patience=50
df = pd.DataFrame((LCA() & {'model_id': model_id}  & {'patience':50} ).fetch())
percentile = np.percentile(df['valid_loss'].astype(float),p)
w_recs = np.stack(df[df['valid_loss']<percentile]['w_rec']).reshape(-1,64)
pca = PCA()
pca.fit(w_recs)

for component in range(64):
    data.append({'Principal component':component,'patience':patience,'variance':np.cumsum(pca.explained_variance_)[component],'ratio':((pca.explained_variance_ratio_))[component],'eigenvalues':((pca.singular_values_))[component]})
df = pd.DataFrame(data)   

# Panel A
#sns.barplot(data=df,x='Principal component',y='ratio',ax=ax0,color='black')
ax0.bar(df['Principal component'],df['ratio'],width=1,align='edge',facecolor='gray', edgecolor='gray')
#sns.scatterplot(data=df,x='Principal component',y='ratio',s=4,ax=ax0,legend=False,color='k')
#ax0.axhline(y=.8,linestyle='--',color='gray')
#ax0.set_xticklabels([])
#ax0.set_yticklabels([])
ax0.set_ylabel('')
ax0.set_xlabel('')
ax0.tick_params(axis='both', which='major', pad=.1,labelsize=6)

# Panel B
sns.heatmap(np.log10(surface),xticklabels=False,yticklabels=False,ax=ax1,cmap='gist_gray',cbar=False)
#ax1.set_xlabel('Tenth PC', fontsize = 6)
#ax1.set_ylabel('First PC', fontsize = 6)
#ax1.set_title('Loss surface', fontsize = 8)


sns.heatmap(pca.components_[0,:].reshape(8,8),xticklabels=False,yticklabels=False,ax=ax2,center=0,cbar=False)
#ax2.set_title('First principal component', fontsize = 6)

sns.heatmap(pca.components_[1,:].reshape(8,8),xticklabels=False,yticklabels=False,ax=ax3,center=0,cbar=False)
#ax3.set_title('Second principal component', fontsize = 6)

#ax4.set_title('Third principal component', fontsize = 6)
sns.heatmap(pca.components_[2,:].reshape(8,8),xticklabels=False,yticklabels=False,ax=ax4,center=0,cbar=True)
cbar = ax4.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)
plt.savefig('/users/langdon/figures/circuitpaperfigures/sloppiness.png',dpi=1000, transparent=True)
# for ax in [ax0,ax1,ax2,ax3,ax4]:
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_xlabel("")
#     ax.set_ylabel("")
#     ax.tick_params(bottom=False)
#     ax.tick_params(left=False)
#     ax.tick_params(axis='both', which='major', pad=.25)

In [133]:
np.cumsum(pca.explained_variance_ratio_[:10])

array([0.27297223, 0.5029712 , 0.64709216, 0.7274459 , 0.76897526,
       0.80493104, 0.8327412 , 0.85599136, 0.8747772 , 0.8903103 ],
      dtype=float32)

# Figure: Perturbation

In [18]:


fig = plt.figure(figsize=(12,8))

gs = gridspec.GridSpec(2,3, figure=fig,width_ratios=[.125,1,1.25],wspace=.25)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])
ax4 = fig.add_subplot(gs[4])
ax5 = fig.add_subplot(gs[5])




query = Model() & {'model_id': model_id}
perturbation = Perturbation(tau=200,
                            sigma_rec=float(0.15),
                            n=query.fetch1('n'))
perturbation.input_layer.weight.data = torch.tensor(query.fetch1('w_in')).float()
perturbation.recurrent_layer.weight.data = torch.tensor(query.fetch1('w_rec')).float()
perturbation.output_layer.weight.data = torch.tensor(query.fetch1('w_out')).float()
perturbation.to(device=device)

# Define trial structure.
t = 3000
dt = 200 * .2
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Generate dataset for simulation

inputs, labels, mask, conditions  = generate_trials(**trial_events,
                                            n_trials=50,
                                            alpha=float(0.2),
                                            tau=perturbation.tau,
                                            sigma_in=.01,
                                            baseline=0.2,
                                            n_coh=6)

q = (LCA() & {'lca_id': lca_id}).fetch1('q')




# Context perturbation
column=0
strength=0
p = q[None,column,:]-q[None,column+1,:]
p = strength * p / np.linalg.norm(p, axis=1)
p = torch.tensor(p).float().to(device=device)

inputs = inputs.to(device)

z, x = perturbation.forward(inputs,p, float(0.2))
z = z.detach().cpu().numpy()

data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')

# motion context
sns.heatmap(motion_df, annot=True,center=0, ax = ax1,yticklabels=motion_df.index.values.round(2),xticklabels=False, cbar=False,linewidths=.025,cmap='Greys')
plt.title('Motion context')
ax1.set_xlabel("")
ax1.set_ylabel("Motion coherence")
ax1.set_title("Motion context")

#color context
sns.heatmap(color_df, annot=True,center=0, ax = ax2,yticklabels=False,xticklabels=False,linewidths=.025,cmap='Greys')
plt.title('Color context')
ax2.set_xlabel("")
ax2.set_ylabel("")
ax2.set_title("Color context")


#perturbation vector
sns.heatmap(p.T,center=0,ax=ax0,cmap='coolwarm',cbar=False)

ax0.set_ylabel("Neuron")
ax0.set_title("Perturbation")


#Motion perturbation
column=2
strength=.02
p = q[None,column,:]-q[None,column+1,:]
p = strength * p / np.linalg.norm(p, axis=1)
p = torch.tensor(p).float().to(device=device)

inputs = inputs.to(device)

z, x = perturbation.forward(inputs,p, float(0.2))
z = z.detach().cpu().numpy()

data = np.concatenate((pd.DataFrame(conditions).values,np.maximum(np.sign(z[:,-1,1,None]-z[:,-1,0,None]),0)),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','output'])

motion_df = df[df['context']=='motion'].groupby(['motion_coh', 'color_coh'])
color_df = df[df['context']=='color'].groupby(['motion_coh', 'color_coh'])

motion_df = ((motion_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')
color_df = ((color_df['output'].apply(np.mean))).reset_index().pivot(index='motion_coh', columns='color_coh', values='output')


sns.heatmap(motion_df,center=0,annot=True, ax = ax4, xticklabels=motion_df.columns.values.round(2),yticklabels=motion_df.index.values.round(2),linewidths=.025,cbar=False,cmap='Greys')
ax4.set_xlabel('Color coherence')
ax4.set_ylabel("Motion coherence")


sns.heatmap(color_df,center=0,annot=True, ax = ax5, xticklabels=motion_df.columns.values.round(2),yticklabels=False,linewidths=.025,cmap='gist_gray_r')
ax5.set_xlabel('Color coherence')
ax5.set_ylabel("")
ax5.set_title("")


#perturbation vector
sns.heatmap(p.T,center=0,ax=ax3,cmap='coolwarm',cbar=False)
ax3.set_ylabel("Neuron")
ax3.set_title("Perturbation")
plt.savefig('/users/langdon/figures/circuitpaperfigures/perturbations.png',dpi=1000, transparent=True)



# Figure: correlations

In [8]:

fig = plt.figure(figsize=(8,1.5))
gs = gridspec.GridSpec(1,4, figure=fig,width_ratios = [1,1,1,1.2])

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])
ax2 = fig.add_subplot(gs[2])
ax3 = fig.add_subplot(gs[3])

# ax3.spines["top"].set_visible(False)
# ax3.spines["right"].set_visible(False)

model_id = "3ZUOEIuF"
lca_id = "zabOLlxl"


w_in = (Model() & {'model_id': model_id}).fetch1('w_in')
w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)

w_in_bar = (LCA & {'lca_id': lca_id}).fetch1('w_in')
w_out_bar = (LCA & {'lca_id': lca_id}).fetch1('w_out')
w_ext_bar = np.hstack((w_in_bar,w_out_bar.T))
w_ext_bar = w_ext_bar / np.linalg.norm(w_ext_bar,axis=0)
sns.heatmap(w_ext.T @ w_ext,ax=ax0,cbar=False,cmap='flare_r')
sns.heatmap(w_ext_bar.T @ w_ext_bar,ax=ax1,cbar=False,cmap='flare_r')



model_id = "x9Yv2etv"
w_in = (Model() & {'model_id': model_id}).fetch1('w_in')
w_out = (Model() & {'model_id': model_id}).fetch1('w_out')
w_ext = np.hstack((w_in,w_out.T))
w_ext = w_ext / np.linalg.norm(w_ext,axis=0)
lca_id = "fDeLRaWE"
w_in_bar = (LCA_unconstrained() & {'lca_id': lca_id}).fetch1('w_in')
w_out_bar = (LCA_unconstrained() & {'lca_id': lca_id}).fetch1('w_out')
w_ext_bar = np.hstack((w_in_bar,w_out_bar.T))
w_ext_bar = w_ext_bar / np.linalg.norm(w_ext_bar,axis=0)
sns.heatmap(w_ext.T @ w_ext,ax=ax2,cbar=False,cmap='flare_r')
sns.heatmap(w_ext_bar.T @ w_ext_bar,ax=ax3,cbar=True,cmap='flare_r')

for ax in [ax0,ax1,ax2,ax3]:
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
#sns.set(fontsize=.2)
cbar = ax3.collections[0].colorbar
# here set the labelsize by 20
cbar.ax.tick_params(labelsize=6)
plt.savefig('/users/langdon/figures/circuitpaperfigures/correlations.png',dpi=1000, transparent=True)

## Figure: schematic

In [86]:
model_id = "3ZUOEIuF"
lca_id = "clcMMw41"
fig = plt.figure(figsize=(4,1.75))
gs = gridspec.GridSpec(1,2, figure=fig)
# RNN connectivity

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1])





#PLOT FITS AND 2D PROJECTIONS
t = 3000
dt = .2 * 200
n_t = int(round(t / dt))
trial_events = {'n_t': int(round(n_t)),
                'cue_on': int(round(n_t * .1)),
                'cue_off': int(round(n_t * .33)),
                'stim_on': int(round(n_t * .4)),
                'stim_off': int(round(n_t)),
                'dec_on': int(round(n_t * .75)),
                'dec_off': int(round(n_t))}

# Load inputs and labels
inputs, labels, mask, conditions = generate_trials(**trial_events,
                                          n_trials=15,
                                          alpha=0.2,
                                          tau=200,
                                          sigma_in=.01,
                                          baseline=0.2,
                                          n_coh=4)

n = (Model() & {'model_id':model_id}).fetch1('n')
size = (Model() & {'model_id':model_id}).fetch1('connectivity')
rnn = RNNModule(connectivity=size, mask=mask, n=n)

rnn.recurrent_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((Model() & {'model_id':model_id}).fetch1('w_out'))
z_mask, z, x = rnn.forward(inputs)
dt=1
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
repeats= x.shape[1]
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
q = (LCA() & {'lca_id':lca_id}).fetch1('q')
x = np.reshape(x,(-1,x.shape[2])) @ q.T
z = np.reshape(z,(-1,z.shape[2]))
conditions = np.repeat(pd.DataFrame(conditions).values, repeats=repeats, axis=0)

data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])
df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df1 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()


# Latent circuit activity
n = 8

rnn = RNNModule(connectivity='small', mask=mask, n=n,sigma_rec=.15)
rnn.recurrent_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_rec'))
rnn.input_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_in'))
rnn.output_layer.weight.data = torch.tensor((LCA() & {'lca_id':lca_id}).fetch1('w_out'))

z_mask, z, x = rnn.forward(inputs)
x = x[:,::dt,:].detach().numpy()
z = z[:,::dt,:].detach().numpy()
time = np.tile(40 * dt * np.arange(x.shape[1]),x.shape[0])[:,None]
x = np.reshape(x,(-1,x.shape[2])) 
z = np.reshape(z,(-1,z.shape[2]))
data = np.concatenate((conditions,time,x,z),1)
df = pd.DataFrame(data=data, columns=['context','motion_coh','color_coh','correct_choice','time','context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'])

df["correct_choice"] = df["correct_choice"].astype(float)
df["time"] = df["time"].astype(float)
df["motion_coh"] = df["motion_coh"].astype(float)
df2 = df.groupby(['context','motion_coh','color_coh','correct_choice','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()

#fig = plt.figure(figsize=(18,3.5))


# for label in (ax5.get_xticklabels() + ax5.get_yticklabels()):
#     label.set_fontsize(fontsize)
# for label in (ax6.get_xticklabels() + ax6.get_yticklabels()):
#     label.set_fontsize(fontsize)
# for label in (ax7.get_xticklabels() + ax7.get_yticklabels()):
#     label.set_fontsize(fontsize)
# for label in (ax8.get_xticklabels() + ax8.get_yticklabels()):
#     label.set_fontsize(fontsize)


markersize=2
markevery=2
lw=1



# Motion subspace
df = df1.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
cohs=np.linspace(-.2,.2,4)
for i in [1,2]:
    ax1.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax1.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='gray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
df = df2.groupby(['context','motion_coh','time'])['context_x_1','context_x_2','motion_x_1','motion_x_2','color_x_1','color_x_2' ,'choice_x_1','choice_x_2','z_1','z_2'].apply(np.mean).reset_index()
for i in [1,2]:
    ax1.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax1.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax0.plot((df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='motion') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)
    ax0.plot((df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_1'])[30:],(df[(df['context']=='color') & (df['motion_coh']==cohs[i])]['motion_x_2'])[30:],color='lightgray',lw=lw,marker='.',markersize=markersize,markevery=markevery)





for ax in [ax0,ax1]:
    ax.tick_params(bottom=False)
    ax.tick_params(left=False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    #ax.spines["top"].set_visible(False)
    #ax.spines["right"].set_visible(False)
    ax.tick_params(axis='both', which='major', pad=1)


plt.savefig('/users/langdon/figures/circuitpaperfigures/schematic_trajectories.png',dpi=1000, transparent=True)

In [76]:
cohs=np.linspace(-.07,.2,4)

In [77]:
cohs

array([-0.2       , -0.06666667,  0.06666667,  0.2       ])